<a href="https://colab.research.google.com/github/shantanu812/MLProjects/blob/main/NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys

assert sys.version_info >= (3, 7)


In [3]:
IS_COLAB = "google.colab" in sys.modules
if IS_COLAB:
    import os
    os.environ["TF_USE_LEGACY_KERAS"] = "1"
    import tf_keras

In [4]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [5]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

In [7]:
from pathlib import Path


In [8]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path=tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

In [9]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)

In [10]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?


In [11]:
vocab_size=1000
max_length=50
text_vec_layer_en=tf.keras.layers.TextVectorization(max_tokens=vocab_size,output_sequence_length=max_length)
text_vec_layer_es=tf.keras.layers.TextVectorization(max_tokens=vocab_size,output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [12]:
text_vec_layer_en.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('the'),
 np.str_('i'),
 np.str_('to'),
 np.str_('you'),
 np.str_('tom'),
 np.str_('a'),
 np.str_('is'),
 np.str_('he')]

In [14]:
text_vec_layer_es.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('startofseq'),
 np.str_('endofseq'),
 np.str_('de'),
 np.str_('que'),
 np.str_('a'),
 np.str_('no'),
 np.str_('tom'),
 np.str_('la')]

In [15]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [16]:
encoder_inputs = tf.keras.layers.Input(shape=[],dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[],dtype=tf.string)

In [17]:
embed_size=128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [18]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [19]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [20]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

In [21]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 89s 25ms/step - loss: 2.9501 - accuracy: 0.4224 - val_loss: 2.2190 - val_accuracy: 0.5163
Epoch 2/10
3125/3125 [==============================] - 59s 19ms/step - loss: 1.8810 - accuracy: 0.5698 - val_loss: 1.6664 - val_accuracy: 0.6096
Epoch 3/10
3125/3125 [==============================] - 58s 19ms/step - loss: 1.4484 - accuracy: 0.6465 - val_loss: 1.4346 - val_accuracy: 0.6510
Epoch 4/10
3125/3125 [==============================] - 60s 19ms/step - loss: 1.2072 - accuracy: 0.6937 - val_loss: 1.3262 - val_accuracy: 0.6745
Epoch 5/10
3125/3125 [==============================] - 58s 19ms/step - loss: 1.0410 - accuracy: 0.7276 - val_loss: 1.2849 - val_accuracy: 0.6819
Epoch 6/10
3125/3125 [==============================] - 60s 19ms/step - loss: 0.9066 - accuracy: 0.7563 - val_loss: 1.2800 - val_accuracy: 0.6865
Epoch 7/10
3125/3125 [==============================] - 59s 19ms/step - loss: 0.7903 - accuracy: 0.7822 - val_loss: 1.2939 -

In [22]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

In [23]:
translate("I like to eat pizza")

1/1 [==============================] - 0s 33ms/step


'me gusta comer [UNK]'

In [24]:
translate("Have you ever done this")

1/1 [==============================] - 0s 19ms/step


'has hecho esto alguna cosa'

In [26]:
tf.random.set_seed(42)
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True))

In [27]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)

In [28]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 111s 31ms/step - loss: 2.0636 - accuracy: 0.5561 - val_loss: 1.4676 - val_accuracy: 0.6467
Epoch 2/10
3125/3125 [==============================] - 74s 24ms/step - loss: 1.2798 - accuracy: 0.6817 - val_loss: 1.2753 - val_accuracy: 0.6809
Epoch 3/10
3125/3125 [==============================] - 72s 23ms/step - loss: 1.0676 - accuracy: 0.7234 - val_loss: 1.2023 - val_accuracy: 0.6986
Epoch 4/10
3125/3125 [==============================] - 73s 24ms/step - loss: 0.9219 - accuracy: 0.7541 - val_loss: 1.1793 - val_accuracy: 0.7023
Epoch 5/10
3125/3125 [==============================] - 73s 24ms/step - loss: 0.8026 - accuracy: 0.7797 - val_loss: 1.1836 - val_accuracy: 0.7063
Epoch 6/10
3125/3125 [==============================] - 75s 24ms/step - loss: 0.7012 - accuracy: 0.8028 - val_loss: 1.2066 - val_accuracy: 0.7021
Epoch 7/10
3125/3125 [==============================] - 76s 24ms/step - loss: 0.6141 - accuracy: 0.8237 - val_loss: 1.2446 

In [29]:
translate("I like soccer")

1/1 [==============================] - 0s 22ms/step


'me gusta el fútbol'

In [30]:
def beam_search(sentence_en, beam_width, verbose=False):
    X = np.array([sentence_en])  # encoder input
    X_dec = np.array(["startofseq"])  # decoder input
    y_proba = model.predict((X, X_dec))[0, 0]  # first token's probas
    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # list of best (log_proba, translation)
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]

    # extra code – displays the top first words in verbose mode
    if verbose:
        print("Top first words:", top_translations)

    for idx in range(1, max_length):
        candidates = []
        for log_proba, translation in top_translations:
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # translation is finished, so don't try to extend it
            X = np.array([sentence_en])  # encoder input
            X_dec = np.array(["startofseq " + translation])  # decoder input
            y_proba = model.predict((X, X_dec))[0, idx]  # last token's proba
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append((log_proba + np.log(word_proba),
                                   f"{translation} {word}"))
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # extra code – displays the top translation so far in verbose mode
        if verbose:
            print("Top translations so far:", top_translations)

        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()

In [31]:
sentence_en = "I love cats and dogs"
beam_search(sentence_en, beam_width=3, verbose=True)

1/1 [==============================] - 0s 88ms/step
Top first words: [(np.float32(-0.00048231788), np.str_('me')), (np.float32(-8.851679), np.str_('odio')), (np.float32(-9.0113535), np.str_('yo'))]
1/1 [==============================] - 0s 33ms/step
Top translations so far: [(np.float32(-0.44103795), 'me encanta'), (np.float32(-1.3827684), 'me [UNK]'), (np.float32(-2.3593988), 'me gustan')]
1/1 [==============================] - 0s 22ms/step
Top translations so far: [(np.float32(-1.4603), 'me encanta los'), (np.float32(-1.7360574), 'me [UNK] y'), (np.float32(-2.2112076), 'me encanta el')]
1/1 [==============================] - 0s 23ms/step
Top translations so far: [(np.float32(-1.7452489), 'me [UNK] y los'), (np.float32(-2.0263782), 'me encanta los perros'), (np.float32(-2.3117213), 'me encanta los gatos')]
1/1 [==============================] - 0s 40ms/step
Top translations so far: [(np.float32(-1.7596743), 'me [UNK] y los perros'), (np.float32(-2.108247), 'me encanta los perros y'), 

'me encanta los gatos y perros'

In [32]:
model.save("my_translation_model.keras")